In [141]:
import numpy as np
import scipy.sparse as sparse
import scipy
import scipy.optimize as opt
import matplotlib.pyplot as plt
import generate_constant_matrices as mat_gen
import generate_proj_mat as gen_proj

np.random.seed(0)

In [176]:
N_omega = 401
N_z = 41
omega = np.linspace(-3, 3, N_omega)
z = np.linspace(0, 0.4, N_z)
delta_omega = np.abs(omega[1] - omega[0])
delta_z = np.abs(z[1] - z[0])
vp = 1
a = 1.61/1.13
vs = vp / (1 + 2 * a * vp / z[-1])
delta_v = vp*vs/(vp - vs)

In [189]:
# Can be used as part of test to see if constraint respected. 
# See if the diagonal of the computed constraint is equal to the diagonal of the 
# matrix exponential
beta_vec = 4*delta_omega * np.random.random((2*N_omega,))
beta = np.zeros((N_omega, N_omega))
for i in range(N_omega):
    for j in range(N_omega):
        beta[i][j] = beta_vec[i + j]
delta_k = np.diag((1/delta_v)*1.j*omega)
Q_plus = delta_k + beta
green_f = scipy.linalg.expm(delta_k*z[-1])
W_plus = np.array([scipy.linalg.expm(Q_plus * z[i]) for i in range(N_z)]).reshape(N_z*N_omega, N_omega)
X = np.concatenate((np.concatenate((np.zeros((2*N_z*N_omega, N_omega)), W_plus)), beta))

In [190]:
dynamics, anti_herm_dynamics = gen_proj.matmul_green_f_basic_proj(omega, vp, z, N_omega)

In [191]:
dynamics_matrices, anti_herm_dynamics_matrices = mat_gen.get_dynamics_matrices(omega, vp, z)

In [192]:
real_test_result = delta_z*X.conj().T@dynamics_matrices[-1]@X
imag_test_result = delta_z*X.conj().T@anti_herm_dynamics_matrices[-1]@X
real_green_f = 2*np.real(green_f)
real_prop = 2*np.real(W_plus[(N_z - 1)*N_omega:])
imag_green_f = 2*np.imag(green_f)
imag_prop = 2*np.imag(W_plus[(N_z - 1)*N_omega:])

In [25]:
# Can be used to see if the Hermitian projection is Hermitian and 
# anti-Hermitian is anti-Hermitian
test_normal = anti_herm_dynamics_matrices[-1][(2*N_z + 10)*N_omega:(2*N_z + 11)*N_omega, 3*N_z*N_omega:].toarray()
test_conj = anti_herm_dynamics_matrices[-1][3*N_z*N_omega:, (2*N_z + 10)*N_omega:(2*N_z + 11)*N_omega].conj().T.toarray()

In [202]:
herm_lin_proj = mat_gen.get_proj_lin_basic(N_omega, N_z)
real_dynamics, imag_dynamics = mat_gen.get_dynamics_matrices(omega, vp, z)

In [214]:
lin_term = np.trace(herm_lin_proj[2*N_z + 10]@X + X.conj().T@(herm_lin_proj[2*N_z + 10].conj().T))
green_f = np.diag(np.exp((1/delta_v)*1.j*omega*z[9]))
cst_term = np.trace(green_f + green_f.conj().T)
quad_term = np.trace(X.conj().T@real_dynamics[9]@X)

In [215]:
lin_term, cst_term, quad_term

((317.32454005818175+0j),
 (389.62916600864054+0j),
 (263.73576639684785-4.920868400259959e-15j))